---
title: "Hypothesis Testing: Making Decisions with Data"
subtitle: "From Questions to Statistical Evidence"
author: "Narjes Mathlouthi"
date: today
format: 
  revealjs:
    logo: /img/logo.png
    theme: default
    css: /files/lecture_notes/lecture10/new-style.css
    slide-number: true
    chalkboard: true
    preview-links: auto
    footer: "Understanding Data – Hypothesis Testing Lecture © 2025"
    incremental: false
    transition: slide
    background-transition: fade
    smaller: true
jupyter: pstat5a
execute:
  echo: false
  warning: false
  message: false
---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import scipy.stats as stats
from scipy.stats import norm, t
import warnings
warnings.filterwarnings('ignore')

# Enhanced color palette
colors = {
    'primary': '#3b82f6',
    'secondary': '#f59e0b', 
    'success': '#10b981',
    'danger': '#ef4444',
    'info': '#8b5cf6',
    'warning': '#f97316',
    'light': '#f8fafc',
    'dark': '#1f2937',
    'accent': '#06b6d4'
}

# Set random seed for reproducibility
np.random.seed(42)

# Welcome to Hypothesis Testing {.center}

**Making Decisions with Data: From Questions to Statistical Evidence**

*"The goal is not to eliminate uncertainty, but to make informed decisions despite it"*

---

## 📢 Quick Announcements

:::: {.columns}
::: {.column width="50%"}
### 📝 Quiz 2 Reminder
**When:**  
- 📅 **Date:** Friday, July 25  
- ⏰ **Window:** 7 AM – 12 AM  
- ⏳ **Duration:** 1 hour once started

**Where:** 💻 Online via Canvas
:::

::: {.column width="50%"}
### 📚 Today's Focus
- **Foundation:** Logic of hypothesis testing
- **Practice:** Real examples with Python
- **Skills:** Making statistical decisions
- **Applications:** From medicine to marketing
:::
::::

---

## Learning Journey Today 🎯

:::: {.columns}
::: {.column}
### 🧠 Conceptual Goals
- **Understand the logic** of hypothesis testing
- **Master the language** of statistical decisions
- **Recognize different types** of errors and their consequences
- **Connect to confidence intervals** from last lecture
:::

::: {.column}
### 🛠️ Practical Skills
- **Formulate hypotheses** from research questions
- **Calculate and interpret** p-values correctly
- **Perform hypothesis tests** in Python
- **Make informed decisions** using statistical evidence
- **Communicate results** effectively
:::
::::

---

## What is Hypothesis Testing? 🤔

In [ ]:
# Create an engaging introduction visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("The Question: Does Treatment Work?", "The Data: Patient Outcomes",
                   "The Analysis: Statistical Testing", "The Decision: Evidence-Based Choice"),
    specs=[[{"type": "bar"}, {"type": "box"}],
           [{"type": "scatter"}, {"type": "table"}]]
)

# Scenario: Testing a new medical treatment
# Before/After comparison
treatments = ['Current Treatment', 'New Treatment']
success_rates = [65, 78]
colors_bar = [colors['danger'], colors['success']]

fig.add_trace(go.Bar(
    x=treatments,
    y=success_rates,
    marker_color=colors_bar,
    text=[f'{rate}%' for rate in success_rates],
    textposition='auto',
    name='Success Rate'
), row=1, col=1)

# Sample data visualization
np.random.seed(42)
current_data = np.random.normal(65, 12, 100)
new_data = np.random.normal(78, 15, 100)

fig.add_trace(go.Box(
    y=current_data,
    name='Current',
    marker_color=colors['danger'],
    boxpoints='outliers'
), row=1, col=2)

fig.add_trace(go.Box(
    y=new_data,
    name='New',
    marker_color=colors['success'],
    boxpoints='outliers'
), row=1, col=2)

# Statistical analysis visualization (sampling distribution)
x_vals = np.linspace(-4, 4, 1000)
y_vals = stats.norm.pdf(x_vals, 0, 1)

fig.add_trace(go.Scatter(
    x=x_vals, y=y_vals,
    mode='lines',
    line=dict(color=colors['primary'], width=3),
    name='Null Distribution'
), row=2, col=1)

# Add critical region
critical_x = x_vals[x_vals >= 1.96]
critical_y = y_vals[x_vals >= 1.96]
fig.add_trace(go.Scatter(
    x=np.concatenate([critical_x, [critical_x[-1], 1.96]]),
    y=np.concatenate([critical_y, [0, 0]]),
    fill='toself',
    fillcolor='rgba(239, 68, 68, 0.3)',
    line=dict(color='rgba(0,0,0,0)'),
    name='Rejection Region'
), row=2, col=1)

# Add observed test statistic
obs_stat = 2.5
fig.add_trace(go.Scatter(
    x=[obs_stat, obs_stat],
    y=[0, max(y_vals)],
    mode='lines',
    line=dict(color=colors['warning'], width=3, dash='solid'),
    name=f'Observed = {obs_stat}',
    showlegend=False
), row=2, col=1)

# Decision table
decision_data = [
    ["Component", "Our Example"],
    ["Research Question", "Does new treatment work better?"],
    ["Null Hypothesis (H₀)", "No difference (μ₁ = μ₂)"],
    ["Alternative (H₁)", "New treatment is better (μ₁ < μ₂)"],
    ["Test Statistic", "t = 2.5"],
    ["P-value", "p = 0.006"],
    ["Decision", "Reject H₀ - Evidence supports new treatment"]
]

fig.add_trace(go.Table(
    header=dict(
        values=["Component", "Our Example"],
        fill_color=colors['info'],
        font=dict(size=12, color='white'),
        align="left"
    ),
    cells=dict(
        values=list(zip(*decision_data[1:])),
        fill_color=[['#f8fafc']*6, ['#e0e7ff']*6],
        font=dict(size=11),
        align="left",
        height=25
    )
), row=2, col=2)

fig.update_layout(
    height=600,
    showlegend=False
    # title_text="Hypothesis Testing: The Complete Picture"
)

# Update axes
fig.update_xaxes(title_text="Treatment Type", row=1, col=1)
fig.update_yaxes(title_text="Success Rate (%)", row=1, col=1)
fig.update_yaxes(title_text="Outcome Score", row=1, col=2)
fig.update_xaxes(title_text="Test Statistic", row=2, col=1)
fig.update_yaxes(title_text="Probability Density", row=2, col=1)

fig.show()

**Hypothesis testing helps us answer:** *"Is what we observed in our sample strong enough evidence to conclude something about the population?"*

---

## The Courtroom Analogy ⚖️

In [ ]:
# Create courtroom analogy visualization
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Criminal Justice System", "Statistical Hypothesis Testing"),
    specs=[[{"type": "table"}, {"type": "table"}]]
)

# Courtroom data
courtroom_data = [
    ["Aspect", "Criminal Court"],
    ["Starting Position", "Defendant is INNOCENT"],
    ["Burden of Proof", "Prosecution must prove GUILT"],
    ["Evidence Standard", "Beyond reasonable doubt"],
    ["Decision Options", "GUILTY or NOT GUILTY"],
    ["Type I Error", "Convict innocent person"],
    ["Type II Error", "Acquit guilty person"],
    ["Consequences", "Balance justice vs. protecting innocent"]
]

# Statistics data
stats_data = [
    ["Aspect", "Hypothesis Testing"],
    ["Starting Position", "NULL hypothesis is TRUE"],
    ["Burden of Proof", "Data must prove ALTERNATIVE"],
    ["Evidence Standard", "p < α (usually 0.05)"],
    ["Decision Options", "REJECT H₀ or FAIL TO REJECT H₀"],
    ["Type I Error", "Reject true null hypothesis"],
    ["Type II Error", "Fail to reject false null"],
    ["Consequences", "Balance discovery vs. false claims"]
]

fig.add_trace(go.Table(
    header=dict(
        values=["Aspect", "Criminal Court"],
        fill_color=colors['danger'],
        font=dict(size=12, color='white'),
        align="left"
    ),
    cells=dict(
        values=list(zip(*courtroom_data[1:])),
        fill_color=[['#fef2f2']*7, ['#fee2e2']*7],
        font=dict(size=10),
        align="left",
        height=30
    )
), row=1, col=1)

fig.add_trace(go.Table(
    header=dict(
        values=["Aspect", "Hypothesis Testing"],
        fill_color=colors['primary'],
        font=dict(size=12, color='white'),
        align="left"
    ),
    cells=dict(
        values=list(zip(*stats_data[1:])),
        fill_color=[['#f8fafc']*7, ['#dbeafe']*7],
        font=dict(size=10),
        align="left",
        height=30
    )
), row=1, col=2)

fig.update_layout(
    height=400,
    showlegend=False
    # title_text="Understanding Hypothesis Testing Through Analogy"
)

fig.show()

**Key Insight:** Just like in court, we never "prove" innocence or "accept" the null hypothesis. We only determine if there's sufficient evidence to reject it!

---

## The Six Steps of Hypothesis Testing 📋

In [ ]:
# Create a comprehensive six-steps visualization
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=("1. State Hypotheses", "2. Choose α Level", "3. Check Assumptions",
                   "4. Calculate Test Statistic", "5. Find P-value", "6. Make Decision"),
    specs=[[{"type": "table"}, {"type": "bar"}, {"type": "scatter"}],
           [{"type": "scatter"}, {"type": "scatter"}, {"type": "table"}]]
)

# Step 1: Hypotheses examples
hyp_data = [
    ["Research Question", "Hypotheses"],
    ["Does drug reduce blood pressure?", "H₀: μ = μ₀ vs H₁: μ < μ₀"],
    ["Are two groups different?", "H₀: μ₁ = μ₂ vs H₁: μ₁ ≠ μ₂"],
    ["Is proportion greater than 50%?", "H₀: p ≤ 0.5 vs H₁: p > 0.5"]
]

fig.add_trace(go.Table(
    header=dict(
        values=["Research Question", "Hypotheses"],
        fill_color=colors['success'],
        font=dict(size=10, color='white'),
        align="left"
    ),
    cells=dict(
        values=list(zip(*hyp_data[1:])),
        fill_color=[['#f0fdf4']*3, ['#dcfce7']*3],
        font=dict(size=9),
        align="left",
        height=25
    )
), row=1, col=1)

# Step 2: Alpha levels
alpha_levels = [0.01, 0.05, 0.10]
alpha_labels = ['Very Conservative\n(α = 0.01)', 'Standard\n(α = 0.05)', 'Liberal\n(α = 0.10)']
alpha_colors = [colors['info'], colors['primary'], colors['warning']]

fig.add_trace(go.Bar(
    x=alpha_labels,
    y=alpha_levels,
    marker_color=alpha_colors,
    text=[f'{a:.2f}' for a in alpha_levels],
    textposition='auto',
    name='Alpha Levels'
), row=1, col=2)

# Step 3: Assumption checking (normality example)
np.random.seed(42)
normal_data = np.random.normal(0, 1, 100)
skewed_data = np.random.exponential(1, 100) - 1

fig.add_trace(go.Histogram(
    x=normal_data,
    name="Normal Data ✓",
    marker_color=colors['success'],
    opacity=0.7,
    nbinsx=20
), row=1, col=3)

fig.add_trace(go.Histogram(
    x=skewed_data,
    name="Skewed Data ⚠",
    marker_color=colors['warning'],
    opacity=0.7,
    nbinsx=20
), row=1, col=3)

# Step 4: Test statistic calculation
x_test = np.linspace(-4, 4, 1000)
y_test = stats.norm.pdf(x_test, 0, 1)

fig.add_trace(go.Scatter(
    x=x_test, y=y_test,
    mode='lines',
    line=dict(color=colors['primary'], width=3),
    name='Standard Normal'
), row=2, col=1)

test_stat = 2.1
# Add vertical line manually
fig.add_trace(go.Scatter(
    x=[test_stat, test_stat],
    y=[0, stats.norm.pdf(test_stat, 0, 1)],
    mode='lines',
    line=dict(color=colors['danger'], width=3, dash='solid'),
    name=f'Test Statistic = {test_stat}',
    showlegend=False
), row=2, col=1)

# Step 5: P-value visualization
p_val_x = x_test[x_test >= test_stat]
p_val_y = y_test[x_test >= test_stat]
fig.add_trace(go.Scatter(
    x=np.concatenate([p_val_x, [p_val_x[-1], test_stat]]),
    y=np.concatenate([p_val_y, [0, 0]]),
    fill='toself',
    fillcolor='rgba(239, 68, 68, 0.3)',
    line=dict(color='rgba(0,0,0,0)'),
    name='P-value Area'
), row=2, col=2)

fig.add_trace(go.Scatter(
    x=x_test, y=y_test,
    mode='lines',
    line=dict(color=colors['primary'], width=3),
    name='Distribution'
), row=2, col=2)

p_value = 1 - stats.norm.cdf(test_stat)
# Add vertical line manually
fig.add_trace(go.Scatter(
    x=[test_stat, test_stat],
    y=[0, stats.norm.pdf(test_stat, 0, 1)],
    mode='lines',
    line=dict(color=colors['danger'], width=3, dash='solid'),
    name=f'p = {p_value:.3f}',
    showlegend=False
), row=2, col=2)

# Step 6: Decision table
decision_data = [
    ["Condition", "Decision", "Conclusion"],
    ["p ≤ α", "Reject H₀", "Statistically significant"],
    ["p > α", "Fail to reject H₀", "Not statistically significant"],
    ["", "Never 'accept H₀'", "Absence of evidence ≠ evidence of absence"]
]

fig.add_trace(go.Table(
    header=dict(
        values=["Condition", "Decision", "Conclusion"],
        fill_color=colors['info'],
        font=dict(size=10, color='white'),
        align="center"
    ),
    cells=dict(
        values=list(zip(*decision_data[1:])),
        fill_color=[['#faf5ff']*3, ['#e9d5ff']*3, ['#ddd6fe']*3],
        font=dict(size=9),
        align="center",
        height=25
    )
), row=2, col=3)

fig.update_layout(
    height=600,
    showlegend=False
    #title_text="The Six Steps of Hypothesis Testing"
)

# Update specific axes
fig.update_yaxes(title_text="α Level", row=1, col=2)
fig.update_xaxes(title_text="Data Value", row=1, col=3)
fig.update_yaxes(title_text="Frequency", row=1, col=3)
fig.update_xaxes(title_text="z-score", row=2, col=1)
fig.update_yaxes(title_text="Density", row=2, col=1)
fig.update_xaxes(title_text="z-score", row=2, col=2)
fig.update_yaxes(title_text="Density", row=2, col=2)

fig.show()

---

## Types of Alternative Hypotheses 🎯

In [ ]:
# Create comprehensive visualization of test types
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=("Two-tailed Test (≠)", "Left-tailed Test (<)", "Right-tailed Test (>)",
                   "When to Use Each", "Critical Values", "Real Examples"),
    specs=[[{"type": "scatter"}, {"type": "scatter"}, {"type": "scatter"}],
           [{"type": "table"}, {"type": "bar"}, {"type": "table"}]]
)

# Standard normal distribution
x = np.linspace(-4, 4, 1000)
y = stats.norm.pdf(x, 0, 1)

# Two-tailed test
alpha = 0.05
z_crit_two = stats.norm.ppf(1 - alpha/2)

fig.add_trace(go.Scatter(
    x=x, y=y,
    mode='lines',
    line=dict(color=colors['primary'], width=3),
    name='Standard Normal'
), row=1, col=1)

# Shade both tails
left_tail = x[x <= -z_crit_two]
right_tail = x[x >= z_crit_two]
fig.add_trace(go.Scatter(
    x=np.concatenate([left_tail, [-z_crit_two, left_tail[0]]]),
    y=np.concatenate([y[x <= -z_crit_two], [0, 0]]),
    fill='toself',
    fillcolor='rgba(239, 68, 68, 0.3)',
    line=dict(color='rgba(0,0,0,0)'),
    name='Rejection Region'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=np.concatenate([right_tail, [right_tail[-1], z_crit_two]]),
    y=np.concatenate([y[x >= z_crit_two], [0, 0]]),
    fill='toself',
    fillcolor='rgba(239, 68, 68, 0.3)',
    line=dict(color='rgba(0,0,0,0)'),
    name='Rejection Region'
), row=1, col=1)

# Left-tailed test
z_crit_left = stats.norm.ppf(alpha)

fig.add_trace(go.Scatter(
    x=x, y=y,
    mode='lines',
    line=dict(color=colors['primary'], width=3),
    name='Standard Normal'
), row=1, col=2)

left_only = x[x <= z_crit_left]
fig.add_trace(go.Scatter(
    x=np.concatenate([left_only, [z_crit_left, left_only[0]]]),
    y=np.concatenate([y[x <= z_crit_left], [0, 0]]),
    fill='toself',
    fillcolor='rgba(239, 68, 68, 0.3)',
    line=dict(color='rgba(0,0,0,0)'),
    name='Rejection Region'
), row=1, col=2)

# Right-tailed test
z_crit_right = stats.norm.ppf(1 - alpha)

fig.add_trace(go.Scatter(
    x=x, y=y,
    mode='lines',
    line=dict(color=colors['primary'], width=3),
    name='Standard Normal'
), row=1, col=3)

right_only = x[x >= z_crit_right]
fig.add_trace(go.Scatter(
    x=np.concatenate([right_only, [right_only[-1], z_crit_right]]),
    y=np.concatenate([y[x >= z_crit_right], [0, 0]]),
    fill='toself',
    fillcolor='rgba(239, 68, 68, 0.3)',
    line=dict(color='rgba(0,0,0,0)'),
    name='Rejection Region'
), row=1, col=3)

# When to use each
usage_data = [
    ["Test Type", "When to Use", "Example Question"],
    ["Two-tailed", "Testing for any difference", "Is the mean different from 100?"],
    ["Left-tailed", "Testing for decrease", "Did response time decrease?"],
    ["Right-tailed", "Testing for increase", "Did test scores improve?"]
]

fig.add_trace(go.Table(
    header=dict(
        values=["Test Type", "When to Use", "Example Question"],
        fill_color=colors['success'],
        font=dict(size=10, color='white'),
        align="left"
    ),
    cells=dict(
        values=list(zip(*usage_data[1:])),
        fill_color=[['#f0fdf4']*3, ['#dcfce7']*3, ['#bbf7d0']*3],
        font=dict(size=9),
        align="left",
        height=25
    )
), row=2, col=1)

# Critical values comparison
test_types = ['Two-tailed', 'One-tailed']
critical_vals = [z_crit_two, z_crit_right]
test_colors = [colors['info'], colors['warning']]

fig.add_trace(go.Bar(
    x=test_types,
    y=critical_vals,
    marker_color=test_colors,
    text=[f'{val:.3f}' for val in critical_vals],
    textposition='auto',
    name='Critical Values (α=0.05)'
), row=2, col=2)

# Real examples
examples_data = [
    ["Field", "Example"],
    ["Medicine", "Drug A vs Drug B effectiveness"],
    ["Psychology", "Therapy reduces anxiety scores"],
    ["Marketing", "New ad increases click rates"],
    ["Quality Control", "Process meets specifications"],
    ["Education", "Method improves test scores"]
]

fig.add_trace(go.Table(
    header=dict(
        values=["Field", "Example Application"],
        fill_color=colors['warning'],
        font=dict(size=10, color='white'),
        align="left"
    ),
    cells=dict(
        values=list(zip(*examples_data[1:])),
        fill_color=[['#fffbeb']*5, ['#fef3c7']*5],
        font=dict(size=9),
        align="left",
        height=25
    )
), row=2, col=3)

fig.update_layout(
    height=600,
    showlegend=False
    #title_text="Types of Hypothesis Tests and When to Use Them"
)

# Update axes for the distribution plots
for col in [1, 2, 3]:
    fig.update_xaxes(title_text="Test Statistic", row=1, col=col)
    fig.update_yaxes(title_text="Probability Density", row=1, col=col)

fig.update_yaxes(title_text="Critical Value", row=2, col=2)

fig.show()

---

## Understanding P-values 📊



**Key Message:** P-value tells us "How surprised should we be by this data if H₀ were true?"

---

## Types of Errors: The Trade-off 🎲

In [ ]:
# Create comprehensive error analysis
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("Error Types Matrix", "Real-World Error Consequences",
                   "The α-β Trade-off", "Error Rates by Field"),
    specs=[[{"type": "table"}, {"type": "table"}],
           [{"type": "scatter"}, {"type": "bar"}]]
)

# Error types matrix
error_matrix = [
    ["", "H₀ is Actually True", "H₀ is Actually False"],
    ["Reject H₀", "TYPE I ERROR\n❌ False Positive\n(Probability = α)", "CORRECT DECISION\n✅ True Positive\n(Probability = 1-β = Power)"],
    ["Fail to Reject H₀", "CORRECT DECISION\n✅ True Negative\n(Probability = 1-α)", "TYPE II ERROR\n❌ False Negative\n(Probability = β)"]
]

fig.add_trace(go.Table(
    header=dict(
        values=["Decision ↓ \\ Reality →", "H₀ is Actually True", "H₀ is Actually False"],
        fill_color=colors['primary'],
        font=dict(size=12, color='white'),
        align="center"
    ),
    cells=dict(
        values=[
            ["Reject H₀", "Fail to Reject H₀"],
            ["TYPE I ERROR\n❌ False Positive\n(Probability = α)", "CORRECT DECISION\n✅ True Negative\n(Probability = 1-α)"],
            ["CORRECT DECISION\n✅ True Positive\n(Probability = Power)", "TYPE II ERROR\n❌ False Negative\n(Probability = β)"]
        ],
        fill_color=[
            ['#f8fafc', '#f8fafc'],
            ['#fef2f2', '#f0fdf4'],
            ['#f0fdf4', '#fef2f2']
        ],
        font=dict(size=11),
        align="center",
        height=40
    )
), row=1, col=1)

# Real-world consequences
consequences_data = [
    ["Context", "Type I Error (False Positive)", "Type II Error (False Negative)"],
    ["Medical Testing", "Healthy person diagnosed sick\n→ Unnecessary treatment", "Sick person not diagnosed\n→ Disease progresses"],
    ["Drug Approval", "Ineffective drug approved\n→ Wasted resources, no benefit", "Effective drug rejected\n→ Patients don't get help"],
    ["Criminal Justice", "Innocent person convicted\n→ Injustice, wrong person jailed", "Guilty person acquitted\n→ Criminal remains free"],
    ["Quality Control", "Good product rejected\n→ Waste, customer dissatisfaction", "Defective product accepted\n→ Safety issues, recalls"],
    ["Fire Alarm", "False alarm\n→ Evacuation, disruption", "Miss real fire\n→ Property damage, injuries"]
]

fig.add_trace(go.Table(
    header=dict(
        values=["Context", "Type I Error (False Positive)", "Type II Error (False Negative)"],
        fill_color=[colors['info'], colors['danger'], colors['warning']],
        font=dict(size=10, color='white'),
        align="center"
    ),
    cells=dict(
        values=list(zip(*consequences_data[1:])),
        fill_color=[['#faf5ff']*5, ['#fef2f2']*5, ['#fffbeb']*5],
        font=dict(size=9),
        align="left",
        height=30
    )
), row=1, col=2)

# Alpha-Beta Trade-off
alpha_range = np.linspace(0.01, 0.20, 50)
# Simulate beta for different alpha (inverse relationship)
beta_range = 0.30 - 0.15 * (alpha_range - 0.01) / 0.19

fig.add_trace(go.Scatter(
    x=alpha_range, y=beta_range,
    mode='lines+markers',
    line=dict(color=colors['danger'], width=3),
    marker=dict(size=6),
    name='β (Type II Error Rate)'
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=alpha_range, y=alpha_range,
    mode='lines+markers',
    line=dict(color=colors['primary'], width=3),
    marker=dict(size=6),
    name='α (Type I Error Rate)'
), row=2, col=1)

# Add annotation
fig.add_annotation(
    x=0.1, y=0.2,
    text="As α increases,<br>β decreases<br>(Trade-off!)",
    showarrow=True,
    arrowhead=2,
    arrowcolor=colors['info'],
    font=dict(size=12),
    row=2, col=1
)

# Typical error rates by field
fields = ['Psychology', 'Medicine', 'Physics', 'Economics', 'Engineering']
alpha_typical = [0.05, 0.01, 0.001, 0.05, 0.01]
field_colors = [colors['primary'], colors['danger'], colors['success'], colors['warning'], colors['info']]

fig.add_trace(go.Bar(
    x=fields,
    y=alpha_typical,
    marker_color=field_colors,
    text=[f'α = {a}' for a in alpha_typical],
    textposition='auto',
    name='Typical α Levels'
), row=2, col=2)

fig.update_layout(
    height=800,
    showlegend=True
    #title_text="Understanding Statistical Errors and Their Consequences"
)

# Update axes
fig.update_xaxes(title_text="α (Type I Error Rate)", row=2, col=1)
fig.update_yaxes(title_text="Error Probability", row=2, col=1)
fig.update_xaxes(title_text="Field", row=2, col=2)
fig.update_yaxes(title_text="Typical α Level", row=2, col=2)

fig.show()

**Bottom Line:** There's always a trade-off between Type I and Type II errors. Choose α based on which error is more costly in your context!

---

## Statistical Power: Detecting True Effects 💪

In [ ]:
# Create comprehensive power analysis
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("What is Statistical Power?", "Factors Affecting Power",
                   "Power vs Sample Size", "Power vs Effect Size"),
    specs=[[{"type": "scatter"}, {"type": "table"}],
           [{"type": "scatter"}, {"type": "scatter"}]]
)

# Power visualization
x_power = np.linspace(-4, 6, 1000)
null_dist = stats.norm.pdf(x_power, 0, 1)
alt_dist = stats.norm.pdf(x_power, 2, 1)  # Effect size = 2

fig.add_trace(go.Scatter(
    x=x_power, y=null_dist,
    mode='lines',
    line=dict(color=colors['primary'], width=3),
    name='Null Distribution (H₀: μ = 0)'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=x_power, y=alt_dist,
    mode='lines',
    line=dict(color=colors['danger'], width=3),
    name='Alternative Distribution (H₁: μ = 2)'
), row=1, col=1)

# Critical value
alpha = 0.05
z_crit = stats.norm.ppf(1 - alpha)
fig.add_trace(go.Scatter(
    x=[z_crit, z_crit],
    y=[0, max(null_dist)],
    mode='lines',
    line=dict(color=colors['info'], width=3, dash='dash'),
    name=f'Critical Value = {z_crit:.2f}',
    showlegend=False
), row=1, col=1)

# Shade Type II error (β)
beta_x = x_power[x_power <= z_crit]
beta_y = stats.norm.pdf(beta_x, 2, 1)
fig.add_trace(go.Scatter(
    x=np.concatenate([beta_x, [z_crit, beta_x[0]]]),
    y=np.concatenate([beta_y, [0, 0]]),
    fill='toself',
    fillcolor='rgba(255, 165, 0, 0.3)',
    line=dict(color='rgba(0,0,0,0)'),
    name='β (Type II Error)'
), row=1, col=1)

# Shade Power (1-β)
power_x = x_power[x_power >= z_crit]
power_y = stats.norm.pdf(power_x, 2, 1)
fig.add_trace(go.Scatter(
    x=np.concatenate([power_x, [power_x[-1], z_crit]]),
    y=np.concatenate([power_y, [0, 0]]),
    fill='toself',
    fillcolor='rgba(16, 185, 129, 0.3)',
    line=dict(color='rgba(0,0,0,0)'),
    name='Power (1-β)'
), row=1, col=1)

# Factors affecting power
power_factors = [
    ["Factor", "Effect on Power", "How to Increase Power"],
    ["Effect Size", "Larger effect → Higher power", "Choose meaningful differences"],
    ["Sample Size", "Larger n → Higher power", "Collect more data"],
    ["Alpha Level", "Larger α → Higher power", "Use α = 0.10 instead of 0.05"],
    ["Variability", "Less noise → Higher power", "Control confounding variables"],
    ["Test Type", "One-tailed → Higher power", "Use directional hypothesis if justified"]
]

fig.add_trace(go.Table(
    header=dict(
        values=["Factor", "Effect on Power", "How to Increase Power"],
        fill_color=colors['success'],
        font=dict(size=11, color='white'),
        align="left"
    ),
    cells=dict(
        values=list(zip(*power_factors[1:])),
        fill_color=[['#f0fdf4']*5, ['#dcfce7']*5, ['#bbf7d0']*5],
        font=dict(size=10),
        align="left",
        height=25
    )
), row=1, col=2)

# Power vs Sample Size
sample_sizes = np.arange(5, 101, 5)
effect_size = 0.5
alpha = 0.05
power_values = []

for n in sample_sizes:
    delta = effect_size * np.sqrt(n)
    t_crit = stats.t.ppf(1 - alpha, n - 1)
    power = 1 - stats.t.cdf(t_crit - delta, n - 1)
    power_values.append(power)

fig.add_trace(go.Scatter(
    x=sample_sizes, y=power_values,
    mode='lines+markers',
    line=dict(color=colors['primary'], width=3),
    marker=dict(size=6),
    name='Power vs Sample Size'
), row=2, col=1)

# Add target power line
target_power_x = np.linspace(min(sample_sizes), max(sample_sizes), 100)
target_power_y = [0.8] * len(target_power_x)
fig.add_trace(go.Scatter(
    x=target_power_x, y=target_power_y,
    mode='lines',
    line=dict(color=colors['danger'], width=2, dash='dash'),
    name='Target Power = 0.8',
    showlegend=False
), row=2, col=1)

# Power vs Effect Size
effect_sizes = np.linspace(0.1, 1.5, 50)
n_fixed = 30
power_effect = []

for es in effect_sizes:
    delta = es * np.sqrt(n_fixed)
    t_crit = stats.t.ppf(1 - alpha, n_fixed - 1)
    power = 1 - stats.t.cdf(t_crit - delta, n_fixed - 1)
    power_effect.append(power)

fig.add_trace(go.Scatter(
    x=effect_sizes, y=power_effect,
    mode='lines+markers',
    line=dict(color=colors['info'], width=3),
    marker=dict(size=6),
    name='Power vs Effect Size'
), row=2, col=2)

# Add target power line
target_power_x2 = np.linspace(min(effect_sizes), max(effect_sizes), 100)
target_power_y2 = [0.8] * len(target_power_x2)
fig.add_trace(go.Scatter(
    x=target_power_x2, y=target_power_y2,
    mode='lines',
    line=dict(color=colors['danger'], width=2, dash='dash'),
    name='Target Power = 0.8',
    showlegend=False
), row=2, col=2)

# Add Cohen's effect size guidelines
effect_lines = [0.2, 0.5, 0.8]
effect_labels = ["Small", "Medium", "Large"]
for i, (line_x, label) in enumerate(zip(effect_lines, effect_labels)):
    fig.add_trace(go.Scatter(
        x=[line_x, line_x],
        y=[0, 1],
        mode='lines',
        line=dict(color=colors['warning'], width=1, dash='dot'),
        name=label,
        showlegend=False
    ), row=2, col=2)

fig.update_layout(
    height=800,
    showlegend=False
    #title_text="Statistical Power: The Ability to Detect True Effects"
)

# Update axes
fig.update_xaxes(title_text="Test Statistic", row=1, col=1)
fig.update_yaxes(title_text="Probability Density", row=1, col=1)
fig.update_xaxes(title_text="Sample Size", row=2, col=1)
fig.update_yaxes(title_text="Statistical Power", row=2, col=1)
fig.update_xaxes(title_text="Effect Size (Cohen's d)", row=2, col=2)
fig.update_yaxes(title_text="Statistical Power", row=2, col=2)

fig.show()

**Key Insight:** Higher power means you're more likely to detect a true effect when it exists. Aim for power ≥ 0.80!

---

## Example 1: One-Sample t-test 📊

### Problem Setup

**Research Question:** A new study technique claims to improve test scores. The current average is 75. We test 25 students using the new method.

In [ ]:
# Generate realistic sample data
np.random.seed(42)
current_average = 75
new_method_scores = np.random.normal(78.5, 8, 25)  # New method might be better

print("📊 Sample Data Summary:")
print("=" * 40)
print(f"Sample size (n): {len(new_method_scores)}")
print(f"Sample mean (x̄): {np.mean(new_method_scores):.2f}")
print(f"Sample std (s): {np.std(new_method_scores, ddof=1):.2f}")
print(f"Current average (μ₀): {current_average}")

### Complete Hypothesis Test

In [ ]:
# Create comprehensive example visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("Sample Data Distribution", "Hypothesis Test Setup",
                   "Test Statistic and P-value", "Decision and Interpretation"),
    specs=[[{"type": "histogram"}, {"type": "table"}],
           [{"type": "scatter"}, {"type": "table"}]]
)

# Sample data histogram
fig.add_trace(go.Histogram(
    x=new_method_scores,
    name="New Method Scores",
    marker_color=colors['primary'],
    opacity=0.7,
    nbinsx=15
), row=1, col=1)

# Add vertical lines manually
sample_mean = np.mean(new_method_scores)
fig.add_trace(go.Scatter(
    x=[sample_mean, sample_mean],
    y=[0, 6],  # Adjust height as needed
    mode='lines',
    line=dict(color=colors['danger'], width=3, dash='solid'),
    name=f'Sample Mean = {sample_mean:.1f}',
    showlegend=False
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=[current_average, current_average],
    y=[0, 6],  # Adjust height as needed
    mode='lines',
    line=dict(color=colors['warning'], width=3, dash='dash'),
    name=f'H₀: μ = {current_average}',
    showlegend=False
), row=1, col=1)

# Test setup table
setup_data = [
    ["Step", "Details"],
    ["1. Hypotheses", "H₀: μ = 75 (no improvement)\nH₁: μ > 75 (improvement)"],
    ["2. Significance Level", "α = 0.05 (5% chance of Type I error)"],
    ["3. Assumptions", "✓ Independence\n✓ Approximate normality (n=25)\n✓ Random sample"],
    ["4. Test Type", "One-sample t-test (σ unknown)"]
]

fig.add_trace(go.Table(
    header=dict(
        values=["Step", "Details"],
        fill_color=colors['info'],
        font=dict(size=11, color='white'),
        align="left"
    ),
    cells=dict(
        values=list(zip(*setup_data[1:])),
        fill_color=[['#faf5ff']*4, ['#e9d5ff']*4],
        font=dict(size=10),
        align="left",
        height=35
    )
), row=1, col=2)

# Calculate test statistic and p-value
x_bar = np.mean(new_method_scores)
s = np.std(new_method_scores, ddof=1)
n = len(new_method_scores)
mu_0 = current_average

t_stat = (x_bar - mu_0) / (s / np.sqrt(n))
df = n - 1
p_value = 1 - stats.t.cdf(t_stat, df)  # Right-tailed test

# t-distribution visualization
x_t = np.linspace(-4, 4, 1000)
y_t = stats.t.pdf(x_t, df)

fig.add_trace(go.Scatter(
    x=x_t, y=y_t,
    mode='lines',
    line=dict(color=colors['primary'], width=3),
    name=f't-distribution (df={df})'
), row=2, col=1)

# Test statistic line
fig.add_trace(go.Scatter(
    x=[t_stat, t_stat],
    y=[0, max(y_t)],
    mode='lines',
    line=dict(color=colors['danger'], width=3, dash='solid'),
    name=f't = {t_stat:.3f}',
    showlegend=False
), row=2, col=1)

# P-value area
pval_x = x_t[x_t >= t_stat]
pval_y = y_t[x_t >= t_stat]
fig.add_trace(go.Scatter(
    x=np.concatenate([pval_x, [pval_x[-1], t_stat]]),
    y=np.concatenate([pval_y, [0, 0]]),
    fill='toself',
    fillcolor='rgba(239, 68, 68, 0.3)',
    line=dict(color='rgba(0,0,0,0)'),
    name=f'P-value = {p_value:.4f}'
), row=2, col=1)

# Critical value
alpha = 0.05
t_crit = stats.t.ppf(1 - alpha, df)
fig.add_trace(go.Scatter(
    x=[t_crit, t_crit],
    y=[0, max(y_t)],
    mode='lines',
    line=dict(color=colors['success'], width=2, dash='dot'),
    name=f'Critical Value = {t_crit:.3f}',
    showlegend=False
), row=2, col=1)

# Results table
results_data = [
    ["Calculation", "Result"],
    ["Test Statistic", f"t = (x̄ - μ₀)/(s/√n) = ({x_bar:.2f} - {mu_0})/{s:.2f}/√{n}) = {t_stat:.3f}"],
    ["Degrees of Freedom", f"df = n - 1 = {n} - 1 = {df}"],
    ["P-value", f"P(T ≥ {t_stat:.3f}) = {p_value:.4f}"],
    ["Critical Value", f"t₀.₀₅,{df} = {t_crit:.3f}"],
    ["Decision", f"{'Reject H₀' if p_value <= alpha else 'Fail to reject H₀'} (p = {p_value:.4f} {'≤' if p_value <= alpha else '>'} α = {alpha})"],
    ["Conclusion", f"{'Strong evidence' if p_value <= alpha else 'Insufficient evidence'} that new method improves scores"]
]

fig.add_trace(go.Table(
    header=dict(
        values=["Calculation", "Result"],
        fill_color=colors['success'] if p_value <= alpha else colors['warning'],
        font=dict(size=11, color='white'),
        align="left"
    ),
    cells=dict(
        values=list(zip(*results_data[1:])),
        fill_color=[['#f0fdf4']*6, ['#dcfce7']*6] if p_value <= alpha else [['#fffbeb']*6, ['#fef3c7']*6],
        font=dict(size=9),
        align="left",
        height=25
    )
), row=2, col=2)

fig.update_layout(
    height=800,
    showlegend=False
    #title_text="Complete One-Sample t-test Example"
)

# Update axes
fig.update_xaxes(title_text="Test Score", row=1, col=1)
fig.update_yaxes(title_text="Frequency", row=1, col=1)
fig.update_xaxes(title_text="t-statistic", row=2, col=1)
fig.update_yaxes(title_text="Probability Density", row=2, col=1)

fig.show()

# Print detailed results
print(f"\n🎯 DETAILED RESULTS:")
print("=" * 50)
print(f"Test Statistic: t = {t_stat:.3f}")
print(f"P-value: {p_value:.4f}")
print(f"Critical Value: {t_crit:.3f}")
print(f"Effect Size (Cohen's d): {(x_bar - mu_0) / s:.3f}")

if p_value <= alpha:
    print(f"\n✅ DECISION: Reject H₀")
    print(f"📊 CONCLUSION: There is statistically significant evidence (p = {p_value:.4f}) that the new study method improves test scores.")
    print(f"💡 PRACTICAL IMPACT: Students using the new method scored {x_bar - mu_0:.1f} points higher on average.")
else:
    print(f"\n❌ DECISION: Fail to reject H₀")
    print(f"📊 CONCLUSION: There is insufficient evidence (p = {p_value:.4f}) that the new study method improves test scores.")
    print(f"💡 PRACTICAL IMPACT: The observed difference could reasonably be due to chance.")

### Using Python's Built-in Functions

In [ ]:
# Method 1: Using scipy.stats
from scipy.stats import ttest_1samp

t_stat_scipy, p_val_scipy = ttest_1samp(new_method_scores, current_average)

print("🐍 PYTHON IMPLEMENTATION:")
print("=" * 40)
print("Method 1: scipy.stats.ttest_1samp")
print(f"t-statistic: {t_stat_scipy:.3f}")
print(f"p-value (two-tailed): {p_val_scipy:.4f}")
print(f"p-value (one-tailed): {p_val_scipy/2:.4f}")

# Method 2: Manual calculation with confidence interval
confidence_level = 0.95
alpha = 1 - confidence_level
t_crit = stats.t.ppf(1 - alpha/2, df)
margin_error = t_crit * (s / np.sqrt(n))
ci_lower = x_bar - margin_error
ci_upper = x_bar + margin_error

print(f"\nMethod 2: Manual with 95% Confidence Interval")
print(f"95% CI: ({ci_lower:.2f}, {ci_upper:.2f})")
print(f"Interpretation: We're 95% confident the true mean is between {ci_lower:.1f} and {ci_upper:.1f}")

# Effect size calculation
cohens_d = (x_bar - mu_0) / s
print(f"\nEffect Size (Cohen's d): {cohens_d:.3f}")
if abs(cohens_d) < 0.2:
    effect_interpretation = "negligible"
elif abs(cohens_d) < 0.5:
    effect_interpretation = "small"
elif abs(cohens_d) < 0.8:
    effect_interpretation = "medium"
else:
    effect_interpretation = "large"
print(f"Effect size interpretation: {effect_interpretation} effect")

---

## Example 2: Two-Sample t-test 📊

### Problem Setup

**Research Question:** Compare effectiveness of two teaching methods

In [ ]:
# Generate realistic data for two groups
np.random.seed(123)
method_A = np.random.normal(75, 10, 30)  # Traditional method
method_B = np.random.normal(82, 12, 28)  # New method

print("📊 TWO-GROUP COMPARISON:")
print("=" * 40)
print("Method A (Traditional):")
print(f"  n = {len(method_A)}, mean = {np.mean(method_A):.2f}, std = {np.std(method_A, ddof=1):.2f}")
print("\nMethod B (New):")
print(f"  n = {len(method_B)}, mean = {np.mean(method_B):.2f}, std = {np.std(method_B, ddof=1):.2f}")
print(f"\nDifference in means: {np.mean(method_B) - np.mean(method_A):.2f} points")

## Statistical vs Practical Significance 🤔

In [ ]:
# Demonstrate the difference between statistical and practical significance
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Large Sample, Small Effect", "Small Sample, Large Effect")
)

# Scenario 1: Large sample, tiny effect
np.random.seed(42)
large_n = 10000
tiny_effect = np.random.normal(100.05, 15, large_n)  # Tiny effect (0.05 points)
t_stat_large, p_val_large = ttest_1samp(tiny_effect, 100)

fig.add_trace(go.Histogram(
    x=tiny_effect,
    name="Large Sample Data",
    marker_color=colors['primary'],
    opacity=0.7,
    nbinsx=50
), row=1, col=1)

# Scenario 1 vertical lines
sample_mean_large = np.mean(tiny_effect)
fig.add_trace(go.Scatter(
    x=[sample_mean_large, sample_mean_large],
    y=[0, 0.025],  # Adjust height as needed
    mode='lines',
    line=dict(color=colors['danger'], width=3, dash='solid'),
    name=f'Mean = {sample_mean_large:.2f}',
    showlegend=False
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=[100, 100],
    y=[0, 0.025],  # Adjust height as needed
    mode='lines',
    line=dict(color=colors['warning'], width=3, dash='dash'),
    name='H₀: μ = 100',
    showlegend=False
), row=1, col=1)

# Scenario 2: Small sample, large effect
small_n = 10
large_effect = np.random.normal(110, 15, small_n)  # Large effect (10 points)
t_stat_small, p_val_small = ttest_1samp(large_effect, 100)

fig.add_trace(go.Histogram(
    x=large_effect,
    name="Small Sample Data",
    marker_color=colors['success'],
    opacity=0.7,
    nbinsx=8
), row=1, col=2)

# Scenario 2 vertical lines
sample_mean_small = np.mean(large_effect)
fig.add_trace(go.Scatter(
    x=[sample_mean_small, sample_mean_small],
    y=[0, 0.02],  # Adjust height as needed
    mode='lines',
    line=dict(color=colors['danger'], width=3, dash='solid'),
    name=f'Mean = {sample_mean_small:.1f}',
    showlegend=False
), row=1, col=2)

fig.add_trace(go.Scatter(
    x=[100, 100],
    y=[0, 0.02],  # Adjust height as needed
    mode='lines',
    line=dict(color=colors['warning'], width=3, dash='dash'),
    name='H₀: μ = 100',
    showlegend=False
), row=1, col=2)

# Add annotations with results
fig.add_annotation(
    x=105, y=0.02,
    text=f"<b>Statistically Significant!</b><br>n = {large_n:,}<br>Effect = {np.mean(tiny_effect) - 100:.2f}<br>t = {t_stat_large:.2f}<br>p = {p_val_large:.2e}<br><br><b>But practically meaningless!</b>",
    showarrow=True,
    arrowhead=2,
    arrowcolor=colors['danger'],
    bgcolor="rgba(255,255,255,0.9)",
    bordercolor=colors['danger'],
    font=dict(size=11),
    row=1, col=1
)

significance_text = "Significant" if p_val_small <= 0.05 else "Not Significant"
fig.add_annotation(
    x=115, y=0.015,
    text=f"<b>{significance_text}</b><br>n = {small_n}<br>Effect = {np.mean(large_effect) - 100:.1f}<br>t = {t_stat_small:.2f}<br>p = {p_val_small:.3f}<br><br><b>But large practical effect!</b>",
    showarrow=True,
    arrowhead=2,
    arrowcolor=colors['success'],
    bgcolor="rgba(255,255,255,0.9)",
    bordercolor=colors['success'],
    font=dict(size=11),
    row=1, col=2
)

fig.update_layout(
    height=500,
    showlegend=False
    #title_text="Statistical Significance ≠ Practical Importance"
)

fig.update_xaxes(title_text="Value", row=1, col=1)
fig.update_yaxes(title_text="Density", row=1, col=1)
fig.update_xaxes(title_text="Value", row=1, col=2)
fig.update_yaxes(title_text="Density", row=1, col=2)

fig.show()

print("🔑 KEY LESSON: Statistical Significance ≠ Practical Importance")
print("=" * 60)
print(f"Left: Tiny effect ({np.mean(tiny_effect) - 100:.2f}) but significant due to large sample")
print(f"Right: Large effect ({np.mean(large_effect) - 100:.1f}) but {'not ' if p_val_small > 0.05 else ''}significant with small sample")
print("\n💡 Always consider BOTH statistical significance AND effect size!")

---

## Best Practices Summary 📋

In [ ]:
# Create best practices visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("Planning Phase", "Analysis Phase", "Interpretation Phase", "Reporting Phase"),
    specs=[[{"type": "table"}, {"type": "table"}],
           [{"type": "table"}, {"type": "table"}]]
)

# Planning best practices
planning_data = [
    ["Best Practice", "Why Important"],
    ["Pre-specify hypotheses", "Avoid p-hacking and bias"],
    ["Choose α before analysis", "Prevent manipulation of results"],
    ["Calculate required sample size", "Ensure adequate power"],
    ["Check assumptions", "Ensure valid results"],
    ["Plan for multiple testing", "Control family-wise error rate"]
]

fig.add_trace(go.Table(
    header=dict(
        values=["Planning Best Practice", "Why Important"],
        fill_color=colors['primary'],
        font=dict(size=11, color='white'),
        align="left"
    ),
    cells=dict(
        values=list(zip(*planning_data[1:])),
        fill_color=[['#f8fafc']*5, ['#dbeafe']*5],
        font=dict(size=10),
        align="left",
        height=25
    )
), row=1, col=1)

# Analysis best practices
analysis_data = [
    ["Best Practice", "Why Important"],
    ["Use appropriate test", "Correct assumptions matter"],
    ["Check diagnostic plots", "Verify model assumptions"],
    ["Report exact p-values", "Avoid arbitrary cutoffs"],
    ["Calculate effect sizes", "Assess practical significance"],
    ["Use robust methods when needed", "Handle assumption violations"]
]

fig.add_trace(go.Table(
    header=dict(
        values=["Analysis Best Practice", "Why Important"],
        fill_color=colors['success'],
        font=dict(size=11, color='white'),
        align="left"
    ),
    cells=dict(
        values=list(zip(*analysis_data[1:])),
        fill_color=[['#f0fdf4']*5, ['#dcfce7']*5],
        font=dict(size=10),
        align="left",
        height=25
    )
), row=1, col=2)

# Interpretation best practices
interpretation_data = [
    ["Best Practice", "Why Important"],
    ["Consider practical significance", "Statistical ≠ meaningful"],
    ["Acknowledge limitations", "Science is about uncertainty"],
    ["Avoid causal language", "Correlation ≠ causation"],
    ["Report confidence intervals", "Show uncertainty in estimates"],
    ["Discuss Type II errors", "Consider power and false negatives"]
]

fig.add_trace(go.Table(
    header=dict(
        values=["Interpretation Best Practice", "Why Important"],
        fill_color=colors['warning'],
        font=dict(size=11, color='white'),
        align="left"
    ),
    cells=dict(
        values=list(zip(*interpretation_data[1:])),
        fill_color=[['#fffbeb']*5, ['#fef3c7']*5],
        font=dict(size=10),
        align="left",
        height=25
    )
), row=2, col=1)

# Reporting best practices
reporting_data = [
    ["Best Practice", "Why Important"],
    ["Report all tests performed", "Prevent selective reporting"],
    ["Include descriptive statistics", "Give complete picture"],
    ["Show data visualizations", "Let data speak"],
    ["Discuss practical implications", "Connect to real world"],
    ["Make data/code available", "Enable reproducibility"]
]

fig.add_trace(go.Table(
    header=dict(
        values=["Reporting Best Practice", "Why Important"],
        fill_color=colors['info'],
        font=dict(size=11, color='white'),
        align="left"
    ),
    cells=dict(
        values=list(zip(*reporting_data[1:])),
        fill_color=[['#faf5ff']*5, ['#e9d5ff']*5],
        font=dict(size=10),
        align="left",
        height=25
    )
), row=2, col=2)

fig.update_layout(
    height=600,
    showlegend=False
    #title_text="Best Practices for Hypothesis Testing"
)

fig.show()

---

## Python Code Templates 💻 {.hidden}

In [ ]:
#| eval: false
#| echo: true

# =============================================================================
# COMPLETE HYPOTHESIS TESTING TOOLKIT
# =============================================================================

import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

# -----------------------------------------------------------------------------
# Template 1: One-Sample t-test
# -----------------------------------------------------------------------------
def one_sample_ttest(data, null_value, alpha=0.05, alternative='two-sided'):
    """
    Perform one-sample t-test with complete analysis
    
    Parameters:
    -----------
    data : array-like
        Sample data
    null_value : float
        Hypothesized population mean
    alpha : float
        Significance level (default 0.05)
    alternative : str
        'two-sided', 'greater', or 'less'
    """
    
    # Calculate statistics
    n = len(data)
    x_bar = np.mean(data)
    s = np.std(data, ddof=1)
    se = s / np.sqrt(n)
    
    # Test statistic
    t_stat = (x_bar - null_value) / se
    df = n - 1
    
    # P-value calculation
    if alternative == 'two-sided':
        p_value = 2 * (1 - stats.t.cdf(abs(t_stat), df))
    elif alternative == 'greater':
        p_value = 1 - stats.t.cdf(t_stat, df)
    elif alternative == 'less':
        p_value = stats.t.cdf(t_stat, df)
    
    # Effect size (Cohen's d)
    cohens_d = (x_bar - null_value) / s
    
    # Confidence interval
    t_crit = stats.t.ppf(1 - alpha/2, df)
    ci_lower = x_bar - t_crit * se
    ci_upper = x_bar + t_crit * se
    
    # Results
    results = {
        'test_statistic': t_stat,
        'p_value': p_value,
        'degrees_of_freedom': df,
        'effect_size': cohens_d,
        'confidence_interval': (ci_lower, ci_upper),
        'reject_null': p_value <= alpha,
        'sample_mean': x_bar,
        'sample_std': s,
        'sample_size': n
    }
    
    return results

# -----------------------------------------------------------------------------
# Template 2: Two-Sample t-test
# -----------------------------------------------------------------------------
def two_sample_ttest(group1, group2, alpha=0.05, equal_var=True):
    """
    Perform two-sample t-test with complete analysis
    """
    
    # Calculate statistics
    n1, n2 = len(group1), len(group2)
    mean1, mean2 = np.mean(group1), np.mean(group2)
    s1, s2 = np.std(group1, ddof=1), np.std(group2, ddof=1)
    
    if equal_var:
        # Pooled variance
        pooled_var = ((n1-1)*s1**2 + (n2-1)*s2**2) / (n1+n2-2)
        se = np.sqrt(pooled_var * (1/n1 + 1/n2))
        df = n1 + n2 - 2
    else:
        # Welch's t-test
        se = np.sqrt(s1**2/n1 + s2**2/n2)
        df = (s1**2/n1 + s2**2/n2)**2 / ((s1**2/n1)**2/(n1-1) + (s2**2/n2)**2/(n2-1))
    
    # Test statistic
    t_stat = (mean1 - mean2) / se
    p_value = 2 * (1 - stats.t.cdf(abs(t_stat), df))
    
    # Effect size (Cohen's d)
    if equal_var:
        pooled_std = np.sqrt(pooled_var)
    else:
        pooled_std = np.sqrt(((n1-1)*s1**2 + (n2-1)*s2**2) / (n1+n2-2))
    
    cohens_d = (mean1 - mean2) / pooled_std
    
    results = {
        'test_statistic': t_stat,
        'p_value': p_value,
        'degrees_of_freedom': df,
        'effect_size': cohens_d,
        'reject_null': p_value <= alpha,
        'group1_stats': {'mean': mean1, 'std': s1, 'n': n1},
        'group2_stats': {'mean': mean2, 'std': s2, 'n': n2}
    }
    
    return results

# -----------------------------------------------------------------------------
# Template 3: Power Analysis
# -----------------------------------------------------------------------------
def power_analysis(effect_size, alpha=0.05, power=0.8):
    """
    Calculate required sample size for desired power
    """
    from scipy.stats import norm
    
    z_alpha = norm.ppf(1 - alpha/2)
    z_beta = norm.ppf(power)
    
    n = ((z_alpha + z_beta) / effect_size) ** 2
    
    return int(np.ceil(n))

# -----------------------------------------------------------------------------
# Example Usage
# -----------------------------------------------------------------------------

# Generate sample data
np.random.seed(42)
sample_data = np.random.normal(105, 15, 25)

# Perform one-sample t-test
results = one_sample_ttest(sample_data, null_value=100, alternative='greater')

print("One-Sample t-test Results:")
print(f"Test statistic: {results['test_statistic']:.3f}")
print(f"P-value: {results['p_value']:.4f}")
print(f"Effect size (d): {results['effect_size']:.3f}")
print(f"95% CI: ({results['confidence_interval'][0]:.2f}, {results['confidence_interval'][1]:.2f})")
print(f"Reject null: {results['reject_null']}")

# Power analysis
required_n = power_analysis(effect_size=0.5, power=0.8)
print(f"\nRequired sample size for d=0.5, power=0.8: {required_n}")

---

## Summary: Key Takeaways 🎯

:::: {.columns}
::: {.column}
### 🧠 Core Concepts
- **Hypothesis testing** provides a framework for making decisions under uncertainty
- **P-values** quantify how surprising our data would be if H₀ were true
- **Statistical significance ≠ practical importance** - always consider effect size
- **Type I and II errors** represent different kinds of mistakes with different costs
- **Power** is the ability to detect true effects when they exist
:::

::: {.column}
### 🛠️ Practical Skills
- **Plan before you analyze** - specify hypotheses and α level in advance
- **Check assumptions** and use appropriate tests
- **Report effect sizes** and confidence intervals, not just p-values
- **Consider practical significance** alongside statistical significance
- **Be honest about limitations** and acknowledge uncertainty
:::
::::



# Thank You! 🎉

**Remember:** Hypothesis testing is a tool for making informed decisions under uncertainty. Use it wisely, report honestly, and always consider the practical implications of your statistical conclusions.

**"The combination of some data and an aching desire for an answer does not ensure that a reasonable answer can be extracted from a given body of data."** - John Tukey

**Next class:** Regression Analysis! 📊


```{=html}
<a href="https://pstat5a.com/schedule.html" class="main-page-btn">🏠 Back to Main Page</a>
```